In [202]:
import pandas as pd

# 讀取財務比例數據，並優先排除超過
financial_ratio = pd.read_csv('/Users/shawn/Github/M1/金融機構與風險管理/New_/S_P500 財務比率_rename_V2.csv')
def remove_columns_with_high_missing_ratio(df, threshold=0.10):
    # 計算每個col缺失值比例
    missing_ratio = df.isnull().mean()
    columns_to_drop = missing_ratio[missing_ratio > threshold].index
    df_dropped = df.drop(columns=columns_to_drop)

    return df_dropped
financial_ratio  = remove_columns_with_high_missing_ratio(financial_ratio, threshold=0.10)


In [203]:
# 讀取rating資料，並更新日期格式
rating = pd.read_csv('/Users/shawn/Github/M1/金融機構與風險管理/New_/rating.csv')

financial_ratio['datadate'] = pd.to_datetime(financial_ratio['datadate'], format='%Y/%m/%d')
financial_ratio['datadate'] = financial_ratio['datadate'].dt.strftime('%Y-%m-%d')


rating['datadate'] = pd.to_datetime(rating['datadate'], format='%Y-%m-%d')
rating['datadate'] = rating['datadate'].dt.strftime('%Y-%m-%d')

In [204]:
merged_df = pd.merge(financial_ratio, rating[['datadate', 'ticker', 'splticrm']], on=['datadate', 'ticker'], how='left')
# merged_df['dividend yield'] = merged_df['dividend yield'].str.rstrip('%').astype('float') / 100


print(merged_df[merged_df['ticker'] == 'AMD'][['datadate','ticker','splticrm']].head()) 
print(rating[rating['ticker'] == 'AMD'][['datadate','ticker','splticrm']].head()) 

        datadate ticker splticrm
3137  2010-01-31    AMD       B-
3138  2010-02-28    AMD       B-
3139  2010-03-31    AMD       B-
3140  2010-04-30    AMD       B-
3141  2010-05-31    AMD       B-
       datadate ticker splticrm
388  2001-01-31    AMD        B
389  2001-02-28    AMD        B
390  2001-03-31    AMD        B
391  2001-04-30    AMD        B
392  2001-05-31    AMD        B


In [205]:
# 以插值法填入
def fill_missing_values_with_interpolation(df, group_col):
    df_filled = df.groupby(group_col).apply(
        lambda group: group.interpolate(method='linear').round(3)
    ).reset_index(drop=True)
    return df_filled
merged_df = fill_missing_values_with_interpolation(merged_df, 'ticker')


In [206]:
# financial_ratio, rating期間範圍無對齊
merged_df.dropna(subset=['splticrm'], inplace=True)
def filter_rows(group):
    # 組內rating前後不同的才會被留下
    mask = group['splticrm'] != group['splticrm'].shift()
    return group[mask]
merged_df = merged_df.groupby('ticker').apply(filter_rows).reset_index(drop=True)
merged_df['ticker'].value_counts().to_frame() 
merged_df = fill_missing_values_with_interpolation(merged_df, 'ticker')


In [212]:
# merged_df保留了所有ticker
merged_df2 = merged_df.groupby('ticker').filter(lambda x: len(x) > 1)
print(len(merged_df2))

merged_df2 = fill_missing_values_with_interpolation(merged_df2, 'ticker')

668


,datadate,ticker,after-tax interest coverage,interest coverage ratio,cash flow/ total debt,operating margin before dep.,operating margin after dep.,return on equity,return on capital employed,LTD/ total assets,...,after-tax return on average common equity,after-tax return on invested capital,after-tax return on average stockholders' equity,gross profit/ total assets,common equity/invested capital,LTD/ invested capital,total debt/ invested capital,capitalization ratio,cash flow margin,splticrm
0,2010-01-31,ABT,11.785,13.248,0.294,0.295,0.230,0.300,0.211,0.322,...,0.300,0.212,0.300,0.422,0.638,0.361,0.496,0.361,0.253,AA
1,2012-10-31,ABT,9.919,16.970,0.251,0.303,0.208,0.106,0.152,0.274,...,0.106,0.058,0.106,0.343,0.667,0.330,0.451,0.331,0.254,A+
2,2017-01-31,ABT,5.940,17.428,0.149,0.238,0.169,0.047,0.118,0.219,...,0.047,0.045,0.047,0.319,0.774,0.221,0.325,0.222,0.116,BBB
3,2010-01-31,ADBE,114.445,214.785,0.467,0.337,0.248,0.080,0.137,0.137,...,0.083,0.080,0.083,0.381,0.830,0.170,0.170,0.170,0.227,BBB+
4,2013-12-31,ADBE,7.617,9.645,0.453,0.233,0.156,0.064,0.079,0.150,...,0.067,0.061,0.067,0.381,0.819,0.181,0.183,0.181,0.184,A-


In [213]:
ratings_map = {
    'D': 0, 'CC': 1, 'CCC': 2, 'CCC+': 3,
    'B-': 5, 'B': 6, 'B+': 7, 'BB-': 8, 'BB': 9, 'BB+': 10,
    'BBB-': 12, 'BBB': 14, 'BBB+': 16,
    'A-': 19, 'A': 22, 'A+': 25, 'AA-': 28, 'AA': 32, 'AA+': 36, 'AAA': 40
}

# 將評級參數化
merged_df2['rating'] = merged_df2['splticrm'].map(ratings_map)

,datadate,ticker,after-tax interest coverage,interest coverage ratio,cash flow/ total debt,operating margin before dep.,operating margin after dep.,return on equity,return on capital employed,LTD/ total assets,...,after-tax return on invested capital,after-tax return on average stockholders' equity,gross profit/ total assets,common equity/invested capital,LTD/ invested capital,total debt/ invested capital,capitalization ratio,cash flow margin,splticrm,rating
0,2010-01-31,ABT,11.785,13.248,0.294,0.295,0.230,0.300,0.211,0.322,...,0.212,0.300,0.422,0.638,0.361,0.496,0.361,0.253,AA,32
1,2012-10-31,ABT,9.919,16.970,0.251,0.303,0.208,0.106,0.152,0.274,...,0.058,0.106,0.343,0.667,0.330,0.451,0.331,0.254,A+,25
2,2017-01-31,ABT,5.940,17.428,0.149,0.238,0.169,0.047,0.118,0.219,...,0.045,0.047,0.319,0.774,0.221,0.325,0.222,0.116,BBB,14
3,2010-01-31,ADBE,114.445,214.785,0.467,0.337,0.248,0.080,0.137,0.137,...,0.080,0.083,0.381,0.830,0.170,0.170,0.170,0.227,BBB+,16
4,2013-12-31,ADBE,7.617,9.645,0.453,0.233,0.156,0.064,0.079,0.150,...,0.061,0.067,0.381,0.819,0.181,0.183,0.181,0.184,A-,19


In [209]:
def process_data(group):
    # 保留前一期的評價
    group['prev_rating'] = group['rating'].shift(1)
    group['rating_diff'] = group['rating'] - group['rating'].shift(1)
    # 變化率
    for col in group.columns:
        if pd.api.types.is_numeric_dtype(group[col]):
            group[col + '_change'] = group[col].pct_change()

    # 刪除前一期評價(prev_rating)
    # group = group[group['is_upgraded'] == 1].drop(columns=['prev_rating'])
    group = group.dropna(subset=['prev_rating'])

    return group

In [210]:
import numpy as np
merged_df3 = merged_df2.groupby('ticker').apply(process_data).reset_index(drop=True)
# 再次處理缺失值與inf
merged_df3 = fill_missing_values_with_interpolation(merged_df3, 'ticker')
cols_with_inf = merged_df3.columns.to_series()[merged_df3.isin([np.inf, -np.inf]).any()]
merged_df3 = merged_df3.drop(cols_with_inf.index, axis=1)
merged_df3 = fill_missing_values_with_interpolation(merged_df3, 'ticker')
merged_df3 = merged_df3.dropna().reset_index(drop=True)




In [211]:
merged_df3.to_csv('/Users/shawn/Github/M1/金融機構與風險管理/New_/SP500_change_V5_rename.csv')